# Context Engineering

Managing context effectively when tools return large results: dataframes, logs, API responses.

In [ ]:
from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.core.context.decorators import context_result

## Tools That Return Large Data

Tools that query databases or read logs often return thousands of rows. Without context management, this overwhelms the model's context window.

In [ ]:
def generate_sales_data(num_rows: int = 500) -> str:
    """Generate sample sales CSV data."""
    import random
    from datetime import datetime, timedelta
    
    products = ["Widget A", "Widget B", "Gadget X", "Gadget Y", "Device Z"]
    regions = ["North", "South", "East", "West"]
    
    lines = ["date,product,region,quantity,price,total"]
    base_date = datetime(2024, 1, 1)
    
    for i in range(num_rows):
        date = (base_date + timedelta(days=i % 365)).strftime("%Y-%m-%d")
        product = random.choice(products)
        region = random.choice(regions)
        qty = random.randint(1, 100)
        price = round(random.uniform(10, 500), 2)
        total = round(qty * price, 2)
        lines.append(f"{date},{product},{region},{qty},{price},{total}")
    
    return "\n".join(lines)

In [ ]:
def generate_log_data(num_lines: int = 300) -> str:
    """Generate sample application log data."""
    import random
    from datetime import datetime, timedelta
    
    levels = ["INFO", "INFO", "INFO", "WARN", "ERROR"]
    components = ["auth", "api", "db", "cache", "worker"]
    messages = {
        "INFO": ["Request processed successfully", "User logged in", "Cache hit", "Task completed"],
        "WARN": ["Slow query detected", "High memory usage", "Rate limit approaching"],
        "ERROR": ["Connection timeout", "Database error", "Authentication failed", "Invalid request"]
    }
    
    lines = []
    base_time = datetime(2024, 6, 15, 10, 0, 0)
    
    for i in range(num_lines):
        ts = (base_time + timedelta(seconds=i * 2)).strftime("%Y-%m-%d %H:%M:%S")
        level = random.choice(levels)
        component = random.choice(components)
        msg = random.choice(messages[level])
        lines.append(f"[{ts}] [{level}] [{component}] {msg}")
    
    return "\n".join(lines)

### The Problem: Large Results Consume Context

Without truncation, tool results fill the context window. The model receives all data but may struggle to extract insights ("the dumb zone").

In [ ]:
def query_sales_raw() -> str:
    """Query sales data without context management."""
    return generate_sales_data(500)

# Check the size of raw data
raw_data = query_sales_raw()
print(f"Raw data: {len(raw_data)} characters, {len(raw_data.split(chr(10)))} rows")

### Solution: The @context_result Decorator

The decorator truncates large results, keeping head and tail rows. The model sees a manageable preview while the full data is saved.

In [ ]:
@context_result()
def query_sales(ctx=None) -> str:
    """Query sales data with automatic truncation."""
    return generate_sales_data(500)

@context_result()
def search_logs(keyword: str, ctx=None) -> str:
    """Search application logs for a keyword."""
    logs = generate_log_data(300)
    matching = [line for line in logs.split("\n") if keyword.upper() in line.upper()]
    return "\n".join(matching) if matching else "No matches found"

In [ ]:
# Decorated function returns truncated preview
truncated = query_sales()
print(f"Truncated result: {len(truncated)} characters")
print("---")
print(truncated)

### Agent with Context-Managed Tools

The agent receives truncated previews instead of full datasets, keeping context usage efficient.

In [ ]:
system_prompt = """You are a data analyst assistant. When analyzing data:
1. Summarize the structure and content from the preview
2. Identify patterns or anomalies visible in the sample
3. Note that full data is saved to the indicated path for detailed analysis"""

agent = get_agent(system_prompt=system_prompt, tools=[query_sales, search_logs])

In [ ]:
prompt = "Query the sales data and summarize what you see."
result, _ = await run_agent(agent, prompt, verbose=True)
print("\n--- Agent Response ---")
print(result.result.output)

In [ ]:
prompt = "Search the logs for ERROR entries and summarize the issues."
result, _ = await run_agent(agent, prompt, verbose=True)
print("\n--- Agent Response ---")
print(result.result.output)

## History Compaction for Long Conversations

Over many turns, conversation history accumulates. `HistoryCompactor` summarizes older exchanges when approaching context limits.

In [ ]:
from agentic_patterns.core.context.history import HistoryCompactor, CompactionConfig
from agentic_patterns.core.agents.utils import nodes_to_message_history

In [ ]:
# Configure with low thresholds for demonstration
config = CompactionConfig(max_tokens=500, target_tokens=200)

async def log_compaction(result):
    print(f"  [COMPACTION] {result.original_messages} msgs ({result.original_tokens} tokens) -> {result.compacted_messages} msgs ({result.compacted_tokens} tokens)")

compactor = HistoryCompactor(config=config, on_compaction=log_compaction)

print(f"Compaction triggers at: {config.max_tokens} tokens")
print(f"Targets reduction to: {config.target_tokens} tokens")

In [ ]:
# Create agent with history processor
agent_with_compaction = get_agent(
    system_prompt="You are a helpful assistant.",
    history_processors=[compactor.create_history_processor()]
)

In [ ]:
# Debug helper to show message contents
def show_messages(label: str, messages: list):
    if not messages:
        print(f"  {label}: (empty)")
        return
    print(f"  {label}: {len(messages)} messages")
    for i, msg in enumerate(messages):
        parts_summary = []
        for part in msg.parts:
            part_type = type(part).__name__
            if hasattr(part, 'content'):
                content = str(part.content)[:50] + "..." if len(str(part.content)) > 50 else str(part.content)
                parts_summary.append(f"{part_type}({content})")
            else:
                parts_summary.append(part_type)
        print(f"    [{i}] {type(msg).__name__}: {', '.join(parts_summary)}")

# Simulate multi-turn conversation
prompts = [
    "Explain what microservices architecture is.",
    "What are the main benefits?",
    "What are common challenges?",
    "How does it compare to monolithic architecture?",
]

message_history = None
for i, prompt in enumerate(prompts, 1):
    print(f"\n{'='*60}")
    print(f"TURN {i}: prompt = '{prompt}'")
    print(f"{'='*60}")
    
    show_messages("BEFORE run_agent (message_history)", message_history)
    
    result, nodes = await run_agent(agent_with_compaction, prompt, message_history=message_history)
    
    print(f"\n  result.new_messages():")
    show_messages("new_messages", result.new_messages())
    
    print(f"\n  compactor._last_processed_messages:")
    show_messages("_last_processed", compactor._last_processed_messages)
    
    message_history = compactor.get_history_for_next_turn(result.new_messages())
    
    print(f"\n  AFTER get_history_for_next_turn:")
    show_messages("next message_history", message_history)
    
    print(f"\n  Response: {result.result.output[:80]}...")